In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from globalModule.globalModule import *

In [10]:
__name__ = "RNN"

# Model

In [11]:
def getModel():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    return model


In [12]:
def getData():
    labelMap = {label:num for num, label in enumerate(actions)}
    sequences, labels = [], []
    for action in actions:
        for sequence in np.array(os.listdir(os.path.join(gesturesDataPath, action))).astype(int):
            window = []
            for frame_num in range(sequenceLen):
                try:
                    res = np.load(os.path.join(gesturesDataPath, action, str(sequence), "{}.npy".format(frame_num)))
                    window.append(res)
                except Exception as e:
                    print(e)
            sequences.append(window)
            labels.append(labelMap[action])
    
    return sequences, labels 

In [13]:
sequences, labels = getData()
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

[Errno 2] No such file or directory: 'data/gestures/click/11/0.npy'
[Errno 2] No such file or directory: 'data/gestures/click/11/1.npy'
[Errno 2] No such file or directory: 'data/gestures/click/11/2.npy'
[Errno 2] No such file or directory: 'data/gestures/click/11/3.npy'
[Errno 2] No such file or directory: 'data/gestures/click/11/4.npy'
[Errno 2] No such file or directory: 'data/gestures/click/11/5.npy'
[Errno 2] No such file or directory: 'data/gestures/click/11/6.npy'
[Errno 2] No such file or directory: 'data/gestures/click/11/7.npy'
[Errno 2] No such file or directory: 'data/gestures/click/11/8.npy'
[Errno 2] No such file or directory: 'data/gestures/click/11/9.npy'
[Errno 2] No such file or directory: 'data/gestures/click/11/10.npy'
[Errno 2] No such file or directory: 'data/gestures/click/11/11.npy'
[Errno 2] No such file or directory: 'data/gestures/click/11/12.npy'
[Errno 2] No such file or directory: 'data/gestures/click/11/13.npy'
[Errno 2] No such file or directory: 'data/g

/var/folders/14/w335m50952gcywsdkh__krl00000gn/T/ipykernel_12109/1839628154.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(sequences)


In [14]:
### INITS

tb_callback = TensorBoard(log_dir=log_dir)

In [15]:
model = getModel()
model.fit(X_train, y_train, epochs=400, callbacks=[tb_callback])

2023-01-08 00:36:36.803105: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [ ]:

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            48896     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 2)                 66        
                                                                 
Total params: 203,426
Trainable params: 203,426
Non-tr

In [ ]:
model.save(f"{modelPath}/{__name__}/test.h5")